<a href="https://colab.research.google.com/github/yallapragada/day_trade_lstm/blob/main/code/iex_stock_prices_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
pd.set_option('display.precision', 3)

In [ ]:
iex_daily_file = r'C:\\uday\\ml_nlp_dsci\\datasets\\iex_finance\\AAPL_SMALL.csv'
stock_prices_df = pd.read_csv(iex_daily_file)
stock_prices_df = stock_prices_df.fillna(method='ffill')
stock_prices_df = stock_prices_df.round({'marketAverage': 3})

In [ ]:
ticker = iex_daily_file.split('\\')[-1].split('.')[0]

In [ ]:
stock_prices_df.head()

,Unnamed: 0,date,label,high,low,average,volume,notional,numberOfTrades,marketHigh,...,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime
0,2019-08-05 09:30:00,2019-08-05,09:30 AM,197.945,196.86,197.515,7013.0,1.385e+06,72.0,198.000,...,197.662,1.315e+06,2.599e+08,3569.0,197.910,197.71,197.99,197.710,0.000,0.000
1,2019-08-05 09:31:00,2019-08-05,09:31 AM,198.380,197.68,198.166,7251.0,1.437e+06,61.0,198.400,...,197.959,2.736e+05,5.416e+07,1332.0,197.715,197.82,197.71,197.845,0.003,0.002
2,2019-08-05 09:32:00,2019-08-05,09:32 AM,198.120,197.72,197.912,2596.0,5.138e+05,30.0,198.260,...,197.936,2.198e+05,4.350e+07,1165.0,197.990,198.12,197.89,198.130,0.002,0.001
3,2019-08-05 09:33:00,2019-08-05,09:33 AM,198.600,197.86,198.214,5530.0,1.096e+06,74.0,198.649,...,198.263,2.490e+05,4.936e+07,1255.0,198.120,198.35,198.13,198.339,0.004,0.003
4,2019-08-05 09:34:00,2019-08-05,09:34 AM,198.400,198.10,198.262,5381.0,1.067e+06,59.0,198.430,...,198.237,1.634e+05,3.239e+07,931.0,198.370,198.10,198.36,198.020,0.004,0.003


In [ ]:
stock_prices_daily_df = stock_prices_df.groupby('date')['label', 'marketAverage']

C:\apps\anaconda3_64\envs\trivia\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
stock_prices_daily_df.head()

,label,marketAverage
0,09:30 AM,197.662
1,09:31 AM,197.959
2,09:32 AM,197.936
3,09:33 AM,198.263
4,09:34 AM,198.237
...,...,...
14402,09:30 AM,246.501
14403,09:31 AM,246.996
14404,09:32 AM,247.596
14405,09:33 AM,247.455


In [ ]:
def replace_dashes(market_averages):
    if '' in market_averages:    
        replacement_value = market_averages[0]
        for market_average in market_averages:
            if (len(str(market_average)) == 0) or (isinstance(market_average, float) is False):
                market_average = replacement_value
            else:
                replacement_value = market_average
    print(market_averages)
    return market_averages

In [ ]:
columns = ['ticker'] + ['date'] + ['price_'+str(i+1) for i in range (0, 120)] + ['price_125'] + ['target']

In [ ]:
def process_single_day_data(date, labels, market_averages):
    
    if len(labels) != len(market_averages):
        print(len(labels), len(market_averages))
        raise Exception('data exception')
        
    if len(labels) < 125:
        raise Exception('insufficient data exception')
    
    df_single_day = pd.DataFrame(columns=columns)
    
    for i in range(len(labels)-125):
        market_averages_120 = market_averages[i:i+120] + [market_averages[i+124]]
        if market_averages[i+124] > market_averages[i+119]:
            target = 1
        else:
            target = 0
        df_single_day.loc[i] = [ticker,date] + market_averages_120 + [target]
        
    return df_single_day 

In [ ]:
df_all_days_data = pd.DataFrame(columns=columns)
bad_days = 0
for date, group in stock_prices_daily_df:
    try:
        print(date)
        process_single_day_data(date, list(group.label), list(group.marketAverage))
        df_single_day = process_single_day_data(date, list(group.label), list(group.marketAverage))
        df_all_days_data = df_all_days_data.append(df_single_day)
    except Exception as e:
        print(e)
        bad_days += 1

2019-05-10
2019-05-28
2019-05-29
2019-05-30
2019-07-09
2019-07-31
2019-08-05
2019-09-13
2019-09-18
2019-10-14
2019-10-22
2019-11-20
2019-12-03
2019-12-10
2019-12-13
2019-12-17
2019-12-26
2020-01-21
2020-01-23
2020-02-06
2020-02-21
2020-02-26
2020-04-01
2020-04-21
2020-04-30
2020-05-21
2020-05-26
2020-06-22
2020-06-25
2020-07-16
2020-07-20
2020-07-22
2020-08-04
2020-08-14
2020-08-25
2020-09-08
2020-10-20
2020-11-09
2020-11-27


In [ ]:
df_all_days_data.tail()

,ticker,date,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,price_113,price_114,price_115,price_116,price_117,price_118,price_119,price_120,price_125,target
80,AAPL_SMALL,2020-11-27,116.894,116.960,117.002,117.021,117.003,116.996,116.970,117.022,...,116.426,116.390,116.387,116.419,116.428,116.427,116.374,116.352,116.326,0
81,AAPL_SMALL,2020-11-27,116.960,117.002,117.021,117.003,116.996,116.970,117.022,117.081,...,116.390,116.387,116.419,116.428,116.427,116.374,116.352,116.370,116.367,0
82,AAPL_SMALL,2020-11-27,117.002,117.021,117.003,116.996,116.970,117.022,117.081,117.092,...,116.387,116.419,116.428,116.427,116.374,116.352,116.370,116.314,116.484,1
83,AAPL_SMALL,2020-11-27,117.021,117.003,116.996,116.970,117.022,117.081,117.092,117.045,...,116.419,116.428,116.427,116.374,116.352,116.370,116.314,116.320,116.531,1
84,AAPL_SMALL,2020-11-27,117.003,116.996,116.970,117.022,117.081,117.092,117.045,116.978,...,116.428,116.427,116.374,116.352,116.370,116.314,116.320,116.249,116.522,1


In [ ]:
iex_daily_out_file = r'C:\\uday\\ml_nlp_dsci\\datasets\\iex_finance\\AAPL_OUT_v3.csv'
df_all_days_data.to_csv(iex_daily_out_file)

In [ ]:
bad_days

0